In [1]:
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
# Represent probabilities of a tier hit
level1 = [1.0, 0, 0, 0, 0]
level2 = [1.0, 0, 0, 0, 0]
level3 = [.65, .3, .5, 0, 0]
level4 = [.5, .35, .15, 0, 0]
level5 = [.37, .35, .25, .03, 0]
level6 = [.245, .35, .3, .10, .005]
level7 = [.2, .3, .33, .15, .02]
level8 = [.15, .25, .35, .2, .05]
level9 = [.1, .15, .35, .3, .1]
levels_df = pd.DataFrame([level1,level2,level3,level4,level5,level6,level7,level8,level9],
                      columns=['Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5'])

# Represent tier numbers & frequencies
tier1 = (12, 39)
tier2 = (12, 26)
tier3 = (12, 21)
tier4 = (9, 13)
tier5 = (6, 10)
tiers_df = pd.DataFrame([tier1,tier2,tier3,tier4,tier5], columns=['Unique Characters', 'Characters in Pool'])

In [3]:
tiers_df

,Unique Characters,Characters in Pool
0,12,39
1,12,26
2,12,21
3,9,13
4,6,10


In [4]:
stats.binom.stats(6,.13, moments='mvsk')

(array(0.78), array(0.6786), array(0.89830702), array(0.47362216))

In [5]:
stats.binom.stats(6,.13)

(array(0.78), array(0.6786))

In [313]:
def hit_prob(level, tier, levels_df=levels_df, tiers_df=tiers_df, removed=0):
    if (1 <= tier <= 5) and (1 <= level <= 9):
        tier_prob = levels_df['Tier {}'.format(tier)][level-1]
        hits_in_pool = tiers_df['Characters in Pool'][tier-1]
        if removed > hits_in_pool: removed=hits_in_pool
        char_hits = hits_in_pool - removed
        chars_in_tier = tiers_df['Unique Characters'][tier-1]
        hit_prob_solo = (char_hits/((chars_in_tier-1)*hits_in_pool + char_hits))*tier_prob
        hit_prob = hit_prob_solo*5 # could adjust this to account for dependent probs of sampling
        return hit_prob
    else:
        return 0.0

#TODO: clean up & account for dependent probs
def multi_hits_prob(level, tiers, levels_df=levels_df, tiers_df=tiers_df, removed=0):
    multi_hits_prob = 0
    for tier in tiers:
        multi_hits_prob += hit_prob(level, tier, levels_df, tiers_df, removed)
    return multi_hits_prob

def multi_rolls_prob(level, tiers, rolls=10, levels_df=levels_df, tiers_df=tiers_df, removed=0):
    multi_rolls_prob = list(map(lambda x:x*multi_hits_prob(level, tiers, levels_df, tiers_df, removed), list(range(1,rolls))))
    return multi_rolls_prob
    

In [137]:
# Interactive plotting with FigureWidget ipywidgets
from plotly import graph_objs as go
from ipywidgets import widgets

In [176]:
level_widget = widgets.IntText(
            value=1.0,
            description='Current Level='
    )

#TODO: make a custom widget that accepts a list of tiers
tiers1 = widgets.IntText(
            value=1.0,
            description='Number of Tier1 Champions='
    )

tiers2 = widgets.IntText(
            value=0.0,
            description='Number of Tier2 Champions='
    )

tiers3 = widgets.IntText(
            value=0.0,
            description='Number of Tier3 Champions='
    )

tiers4 = widgets.IntText(
            value=0.0,
            description='Number of Tier4 Champions='
    )

tiers5 = widgets.IntText(
            value=0.0,
            description='Number of Tier5 Champions='
    )

In [177]:
rolls = 10
initial_level = 1
initial_tiers = []
initial_tiers.extend([1 for x in range(tiers1.value)])
initial_tiers.extend([2 for x in range(tiers2.value)])
initial_tiers.extend([3 for x in range(tiers3.value)])
initial_tiers.extend([4 for x in range(tiers4.value)])
initial_tiers.extend([5 for x in range(tiers5.value)])

In [266]:
#TODO: plot probs of each individual hit, and probs of any hit
total_trace = go.Scatter(x=list(range(1,rolls)), 
                         y=multi_rolls_prob(initial_level, initial_tiers),
                         line={'width':4.0},
                         opacity=0.5,
                        name='Hit on anything')
trace1 = go.Scatter(x=list(range(1,rolls)), 
                    y=multi_rolls_prob(initial_level, [1]),
                   line={'color':'DarkGray'},
                    #opacity=0.9,
                    name='Tier1 hit'
                   )

#TODO: Create additional five traces
g = go.FigureWidget(data=[total_trace, trace1],
             layout=go.Layout(
             title='TFTdex',
                 height=500,
             yaxis={'range':[0,10],
                   'title':'Expected Number of Hits',
                   'nticks':20},
            xaxis={'title':'Number of Rolls'}
             ))

In [269]:
# Function to handle input from widgets and alter graph
def validate():
    if 1 <= level_widget.value <= 9:
        return True
    else:
        return False
    
def response(change):
    if validate():
        level = level_widget.value
        tiers = []
        if tiers1.value <= tiers_df['Unique Characters'][0]: tiers.extend([1]*tiers1.value)
        if tiers2.value <= tiers_df['Unique Characters'][1]: tiers.extend([2]*tiers2.value)
        if tiers3.value <= tiers_df['Unique Characters'][2]: tiers.extend([3]*tiers3.value)
        if tiers4.value <= tiers_df['Unique Characters'][3]: tiers.extend([4]*tiers4.value)
        if tiers5.value <= tiers_df['Unique Characters'][4]: tiers.extend([5]*tiers5.value)
        #print(tiers)

        updated_probs = multi_rolls_prob(level, tiers)
        with g.batch_update():
            g.data[0].y = updated_probs
            g.data[1].y = multi_rolls_prob(level, [1])

#level_widget.observe(response, names='value')
#tiers1.observe(response, names='value')
#tiers2.observe(response, names='value')
#tiers3.observe(response, names='value')
#tiers4.observe(response, names='value')
#tiers5.observe(response, names='value')
g.observe(response, names='value')

In [270]:
#widgets.VBox([container, container1, container2, container3, container4, container5, g])
widgets.VBox([level_widget, tiers1, tiers2, tiers3, tiers4, tiers5, g])

In [314]:
#TODO: Create version with stacked bar graph
initial_level = 1


level_widget = widgets.IntText(
            value=initial_level,
            description='Current Level='
    )

champA = widgets.IntText(
    value=1.0,
    description='Tier of Ath Unit=')
champAgone = widgets.IntText(
    value=0.0,
    description='# of A units removed from pool=')
containerA = widgets.HBox(children=[champA, champAgone])

champB = widgets.IntText(
    value=0.0,
    description='Tier of Bth Unit=')
champBgone = widgets.IntText(
    value=0.0,
    description='# of B units removed from pool=')
containerB = widgets.HBox(children=[champB, champBgone])

champC = widgets.IntText(
    value=0.0,
    description='Tier of Cth Unit=')
champCgone = widgets.IntText(
    value=0.0,
    description='# of C units removed from pool=')
containerC = widgets.HBox(children=[champC, champCgone])

champD = widgets.IntText(
    value=0.0,
    description='Tier of Dth Unit=')
champDgone = widgets.IntText(
    value=0.0,
    description='# of D units removed from pool=')
containerD = widgets.HBox(children=[champD, champDgone])

champE = widgets.IntText(
    value=0.0,
    description='Tier of Eth Unit=')
champEgone = widgets.IntText(
    value=0.0,
    description='# of E units removed from pool=')
containerE = widgets.HBox(children=[champE, champEgone])


In [315]:
# Establish color dict for tiers
color_dict = {0:'Red', 1:'Gray', 2:'Green', 3:'Blue', 4:'Purple', 5:'Gold'}

In [326]:
barA = go.Bar(x=list(range(1,rolls)), 
              y=multi_rolls_prob(level_widget.value, [champA.value], removed=champAgone.value),
              name='Champ A',
              marker={'color':color_dict[champA.value],
                     'line': {'width':1.5}
                     }
             )
barB = go.Bar(x=list(range(1,rolls)), 
              y=multi_rolls_prob(level_widget.value, [champB.value], removed=champBgone.value),
              name='Champ B',
              marker={'color':color_dict[champB.value],
                     'line': {'width':1.5}
                     }
             )
barC = go.Bar(x=list(range(1,rolls)), 
              y=multi_rolls_prob(level_widget.value, [champC.value], removed=champCgone.value),
              name='Champ C',
              marker={'color':color_dict[champC.value],
                     'line': {'width':1.5}
                     }
             )
barD = go.Bar(x=list(range(1,rolls)), 
              y=multi_rolls_prob(level_widget.value, [champD.value], removed=champDgone.value),
              name='Champ D',
              marker={'color':color_dict[champD.value],
                     'line': {'width':1.5}
                     }
             )
barE = go.Bar(x=list(range(1,rolls)), 
              y=multi_rolls_prob(level_widget.value, [champE.value], removed=champEgone.value),
              name='Champ E',
              marker={'color':color_dict[champE.value],
                     'line': {'width':1.5}
                     }
             )

fig = go.FigureWidget(data=[barA, barB, barC, barD, barE],
             layout=go.Layout(
                 title='TFTdex Bar Mode',
                 height=500,
                 barmode='stack',
                 yaxis={'range':[0,6],
                   'title':'Expected Number of Hits',
                   'nticks':8},
                 xaxis={'title':'Number of Rolls'}
             ))

In [327]:
def validate_bar():
    if (1 <= level_widget <=9) and (1 <= champA.value <= 5) and (1 <= champB.value <= 5) and (1 <= champC.value <= 5) and (1 <= champD.value <= 5) and (1 <= champE.value <= 5):
        return True
    else:
        return False

def check(change):
    if validate_bar:
        level = level_widget.value
        with fig.batch_update():
            fig.data[0].y=multi_rolls_prob(level, [champA.value], removed=champAgone.value)
            fig.data[0].marker['color']=color_dict[champA.value]
            fig.data[1].y=multi_rolls_prob(level, [champB.value], removed=champBgone.value)
            fig.data[1].marker['color']=color_dict[champB.value]
            fig.data[2].y=multi_rolls_prob(level, [champC.value], removed=champCgone.value)
            fig.data[2].marker['color']=color_dict[champC.value]
            fig.data[3].y=multi_rolls_prob(level, [champD.value], removed=champDgone.value)
            fig.data[3].marker['color']=color_dict[champD.value]
            fig.data[4].y=multi_rolls_prob(level, [champE.value], removed=champEgone.value)
            fig.data[4].marker['color']=color_dict[champE.value]
 
level_widget.observe(check, names='value')
champA.observe(check, names='value')
champAgone.observe(check, names='value')
champB.observe(check, names='value')
champBgone.observe(check, names='value')
champC.observe(check, names='value')
champCgone.observe(check, names='value')
champD.observe(check, names='value')
champDgone.observe(check, names='value')
champE.observe(check, names='value')
champEgone.observe(check, names='value')
#fig.observe(check, names='value')

In [328]:
widgets.VBox([level_widget, containerA, containerB, containerC, containerD, containerE, fig])

# Calculate the probability of a hit of one unit at a given level

In [49]:
level=5
tier=3
print(hit_prob(level,tier))

0.10416666666666666


# Calculate the probability of a hit of multiple units at a given level

In [50]:
level=5
tiers=[3,2,2]
print(multi_hits_prob(level, tiers))

0.39583333333333326


# Calculate the mean & variance of hits given a certain number of rolls

In [51]:
level=5
tiers=[3,2,2]
rolls=6
one_roll_prob=multi_hits_prob(level,tiers)
mean, var, skew, kurt = stats.binom.stats(rolls, one_roll_prob, moments='mvsk')
print('mean = {} \n var = {}'.format(mean,var))

mean = 2.3749999999999996 
 var = 1.4348958333333333
